In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

file="/content/drive/My Drive/BTP/NGSIM.csv"
us101 = pd.read_csv(file)
us101.sort_values(by=["Global_Time"],inplace=True)
us101=us101.reset_index(drop=True)

full_name = ['front','back','left_front','right_front','left_back','right_back','left','right','front_front']
columns=[['F_Vehicle_ID', 'F_Local_X', 'F_Local_Y', 'F_v_Vel', 'F_v_Acc', 'F_v_long', 'F_x_lat', 'F_y_long', 'F_type']
        ,['B_Vehicle_ID', 'B_Local_X', 'B_Local_Y', 'B_v_Vel', 'B_v_Acc', 'B_v_long', 'B_x_lat', 'B_y_long', 'B_type']
        ,['LF_Vehicle_ID', 'LF_Local_X', 'LF_Local_Y', 'LF_v_Vel', 'LF_v_Acc', 'LF_v_long', 'LF_x_lat', 'LF_y_long', 'LF_type']
        ,['RF_Vehicle_ID', 'RF_Local_X', 'RF_Local_Y', 'RF_v_Vel', 'RF_v_Acc', 'RF_v_long', 'RF_x_lat', 'RF_y_long', 'RF_type']
        ,['LB_Vehicle_ID', 'LB_Local_X', 'LB_Local_Y', 'LB_v_Vel', 'LB_v_Acc', 'LB_v_long', 'LB_x_lat', 'LB_y_long', 'LB_type']
        ,['RB_Vehicle_ID', 'RB_Local_X', 'RB_Local_Y', 'RB_v_Vel', 'RB_v_Acc', 'RB_v_long', 'RB_x_lat', 'RB_y_long', 'RB_type']
        ,['L_Vehicle_ID', 'L_Local_X', 'L_Local_Y', 'L_v_Vel', 'L_v_Acc', 'L_v_long', 'L_x_lat', 'L_y_long', 'L_type']
        ,['R_Vehicle_ID', 'R_Local_X', 'R_Local_Y', 'R_v_Vel', 'R_v_Acc', 'R_v_long', 'R_x_lat', 'R_y_long', 'R_type']
        ,['FF_Vehicle_ID', 'FF_Local_X', 'FF_Local_Y', 'FF_v_Vel', 'FF_v_Acc', 'FF_v_long', 'FF_x_lat', 'FF_y_long', 'FF_type']]

for temp_col in columns:
    us101[temp_col] = pd.DataFrame([[pd.np.NaN]*9], index=us101.index)

# print(us101.columns)

calc_distances = {}

def calc_distance(idx1, idx2):
    # print(idx1, idx2)
    row1 = us101.loc[idx1]
    row2 = us101.loc[idx2]
    try:
        distance = calc_distances[(idx1,idx2)]
    except:
        distance = (row1.Local_X-row2.Local_X)**2+(row1.Local_Y-row2.Local_Y)**2
    return distance

def append_front_back(index,name,nearest_dict,vehicle_row_dict):
    if(index<0):
        return
    front=us101.loc[index].Preceding
    back=us101.loc[index].Following
    if(front != 0):
        nearest_dict[name[0]]=vehicle_row_dict.get(front,-1)
    if(back != 0):
        nearest_dict[name[1]]=vehicle_row_dict.get(back,-1)

def find_nearest(lanes_dict, vehicle_row_dict):
    # print(vehicle_row_dict)
    for lane in [1,2,3,4,5]:
        for vehicle in lanes_dict[lane]:
            nearest_dict={'left_back':-2,'left_front':-2,'right_back':-2,'right_front':-2, 'front':-2,
                            'back':-2, 'right':-2, 'left':-2,'front_front':-2}
            try:
                min_dist = 90000
                for k in lanes_dict[lane-1]:
                    dist = calc_distance(vehicle,k)
                    if dist<min_dist:
                        min_dist = dist
                        nearest_dict['left']=k
            except:
                nearest_dict['left']=-1

            try:
                min_dist = 90000
                for k in lanes_dict[lane+1]:
                    dist = calc_distance(vehicle,k)
                    if dist<min_dist:
                        min_dist = dist
                        nearest_dict['right']=k
            except:
                nearest_dict['right']=-1


            #getting front and back
            append_front_back(vehicle,['front','back'],nearest_dict,vehicle_row_dict)
            #getting left front and back
            append_front_back(nearest_dict['left'],['left_front','left_back'],nearest_dict,vehicle_row_dict)
            #getting right front and back
            append_front_back(nearest_dict['right'],['right_front','right_back'],nearest_dict,vehicle_row_dict)
            #getting front front
            append_front_back(nearest_dict['front'],['front_front','front_back'],nearest_dict,vehicle_row_dict)


            vid_t,loc_x_t,loc_y_t,v_t,a_t=us101.loc[vehicle,['Vehicle_ID','Local_X','Local_Y','v_Vel','v_Acc']].values

            for i in range(len(full_name)):
                
                name=full_name[i]

                if(nearest_dict[name]>0):
                    vid_p,loc_x_p,loc_y_p,v_p,a_p=us101.loc[nearest_dict[name],['Vehicle_ID','Local_X','Local_Y','v_Vel','v_Acc']].values
                    v_long=v_t-v_p
                    x_lat=loc_x_p-loc_x_t
                    x_long=loc_y_p-loc_y_t
                    ttc=x_long/(v_long+0.000001)

                    temp_col=columns[i]
                    df_values=us101.loc[nearest_dict[name],['Vehicle_ID','Local_X','Local_Y','v_Vel','v_Acc']].values.tolist()
                    cal_values=[v_long,x_lat,x_long,ttc]

                    #concatenate list and assign to values
                    us101.loc[vehicle,temp_col] = df_values + cal_values 

    return


time = us101['Global_Time'][0]
lanes_dict = {1:[],2:[],3:[],4:[],5:[]}
vehicle_row_dict = {}
for index in range(len(us101)):
    row=us101.loc[index,:]
    
    if(index%5000==0):
        print("Total rows done : {} time : {}".format(index, current_time))
        

    if row.Global_Time == time:
        vehicle_row_dict[row.Vehicle_ID]=index
        try:
            lanes_dict[row.Lane_ID].append(index)
        except:
            continue
    else:
        find_nearest(lanes_dict, vehicle_row_dict)
        lanes_dict = {1:[],2:[],3:[],4:[],5:[]}
        vehicle_row_dict = {}
        time=row.Global_Time
        vehicle_row_dict[row.Vehicle_ID]=index
        try:
            lanes_dict[row.Lane_ID].append(index)
        except:
            continue

#print(us101.loc[:1000].to_csv())
us101.to_csv("/content/drive/My Drive/BTP/NGSIM_processed.csv",index=None)


Total rows done : 0 time : 10:03:28
Total rows done : 5000 time : 10:10:35
Total rows done : 10000 time : 10:18:06
Total rows done : 15000 time : 10:25:20
Total rows done : 20000 time : 10:32:50
Total rows done : 25000 time : 10:40:07
Total rows done : 30000 time : 10:47:39
Total rows done : 35000 time : 10:55:04
Total rows done : 40000 time : 11:02:22
Total rows done : 45000 time : 11:09:39
Total rows done : 50000 time : 11:16:47
